In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
import os

import ipywidgets as widgets
from ipywidgets import interact, interactive, IntSlider, FloatSlider

In [2]:
def get_img(path: str):
    img = cv.imread(path, cv.IMREAD_GRAYSCALE)
    return img

In [3]:
samples = os.listdir('img')

In [5]:
def gamma_correction(img, gamma):
    invGamma = 1 / gamma

    table = [((i / 255) ** invGamma) * 255 for i in range(256)]
    table = np.array(table, np.uint8)

    return cv.LUT(img, table)

def test_gamma(sample, gamma):
    img = get_img(f'img/{sample}')
    gc = gamma_correction(img, gamma)
    plt.imshow(gc, 'gray')
    
interactive(test_gamma, sample=samples, gamma=FloatSlider(min=0, max=5, value=2.2))

interactive(children=(Dropdown(description='sample', options=('Catalogo_fosseis_PontaGrossa-64.png', 'Catalogo…

In [6]:
def threshold(img, thresh):
    return cv.threshold(img, thresh, 255, cv.THRESH_BINARY)
    
def test_threshold(sample, thresh):
    img = get_img(f'img/{sample}')
    th, dst = threshold(img, thresh)
    fig, axes = plt.subplots(1, 2)
    axes[0].imshow(img, 'gray')
    axes[1].imshow(dst, 'gray')
    
interactive(test_threshold, sample=samples, thresh=IntSlider(min=0, max=255, value=100))

interactive(children=(Dropdown(description='sample', options=('Catalogo_fosseis_PontaGrossa-64.png', 'Catalogo…

In [7]:
def edges(img, minimum, maximum, aperture, l2_gradient):
    return cv.Canny(img, minimum, maximum, apertureSize = aperture, L2gradient=l2_gradient)

#     plt.subplot(121), plt.imshow(img, 'gray')
#     plt.xticks([]), plt.yticks([])
#     plt.title('Imagem original')

#     plt.subplot(122), plt.imshow(edges, 'gray')
#     plt.xticks([]), plt.yticks([])
#     plt.title('Detecção de bordas')

#     plt.show()
    
def test_edges(sample, minimum, maximum, aperture, l2_gradient):
    img = get_img(f'img/{sample}')
    edg = edges(img, minimum, maximum, aperture, l2_gradient)
    plt.imshow(edg, 'gray')
    
interact(test_edges, sample=samples, minimum=(0, 255), maximum=(0, 255), aperture=IntSlider(min=3, max=7, step=2, value=3), l2_gradient=False)

interactive(children=(Dropdown(description='sample', options=('Catalogo_fosseis_PontaGrossa-64.png', 'Catalogo…

<function __main__.test_edges(sample, minimum, maximum, aperture, l2_gradient)>

In [9]:
def invert(img):
    return (255-img)

def generate(sample, thresh, gamma, edge_min, edge_max, aperture, l2_gradient):
    img = get_img(f'img/{sample}')
    gamma = gamma_correction(img, gamma)
    th, dst = threshold(gamma, thresh)    
    edg = edges(gamma, edge_min, edge_max, aperture, l2_gradient)
    
    plt.subplot(221)
    plt.title('Imagem Original')
    plt.xticks([]), plt.yticks([])
    plt.imshow(img, 'gray')
    
    plt.subplot(222)
    plt.title('Gama aplicado')
    plt.xticks([]), plt.yticks([])
    plt.imshow(gamma, 'gray')
    
    plt.subplot(223)
    plt.title('Gama + Threshold (Invertido)')
    plt.xticks([]), plt.yticks([])
    plt.imshow(invert(dst), 'gray')
    
    plt.subplot(224)
    plt.title('Gama + Canny')
    plt.xticks([]), plt.yticks([])
    plt.imshow(edg, 'gray')
    
interact(
    generate,
    sample=samples,
    thresh=(0, 255),
    gamma=FloatSlider(min=0.01, max=5, value=2.2),
    edge_min=(0, 255),
    edge_max=(0, 255),
    aperture=IntSlider(min=3, max=7, step=2, value=3),
    l2_gradient=True
)

interactive(children=(Dropdown(description='sample', options=('Catalogo_fosseis_PontaGrossa-64.png', 'Catalogo…

<function __main__.generate(sample, thresh, gamma, edge_min, edge_max, aperture, l2_gradient)>